
1) PCA to reduce dimensionality of trial-averaged event-related responses (rois x time), with respect to time dimension. Intuitive concept: each ROI's datapoint resides in n dimensional space where n is the number of samples in the event-related window. PCA finds new set of (orthogonal) axes that maximizes the variance in the activity. These new axes are linear combinations of the original axes

2) Spectral clustering: The roi data are now characterized by a reduced set of optimized axes describing time. We now cluster using spectral clustering, which does not assume any particular shape of the cluster data points (eg. kmeans assumes data clouds are gaussian). The three main steps of spectral clustering are **A)** create graph theory similarity matrix for each ROI based on how close other ROIs are in the PCA space, **B)** perform eigendecomposition of the similarity matrix, **C)** Use kmeans clustering on the transformed data. 

In [ ]:
import pickle
import math
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, SVR, LinearSVC
from sklearn.metrics import accuracy_score, silhouette_score, adjusted_rand_score, silhouette_samples
from sklearn.cluster import AgglomerativeClustering, SpectralClustering, KMeans
from sklearn.model_selection import KFold, LeaveOneOut, train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.kernel_ridge import KernelRidge
from sklearn import linear_model
from sklearn.manifold import TSNE
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from patsy import (ModelDesc, EvalEnvironment, Term, EvalFactor, LookupFactor, dmatrices, INTERCEPT)
from statsmodels.distributions.empirical_distribution import ECDF
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.colorbar as colorbar
import sys
import os
import re
import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import json
import utils

In [ ]:
def standardize_plot_graphics(ax):
    [i.set_linewidth(0.5) for i in ax.spines.itervalues()]
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    return ax

def fit_regression(x, y):
    lm = sm.OLS(y, sm.add_constant(x)).fit()
    x_range = sm.add_constant(np.array([x.min(), x.max()]))
    x_range_pred = lm.predict(x_range)
    return lm.pvalues[1], lm.params[1], x_range[:,1], x_range_pred, lm.rsquared

def CDFplot(x, ax, **kwargs):
    x = np.array(x)
    ix=np.argsort(x)
    ax.plot(x[ix], ECDF(x)(x)[ix], **kwargs)
    return ax


def fit_regression_and_plot(x, y, ax, plot_label='', color='k', linecolor='r', markersize=3,
                            show_pval=True):
    #linetype is a string like 'bo'
    pvalue, slope, temp, temppred, R2 = fit_regression(x, y)   
    if show_pval:
        plot_label = '%s p=%.2e\nr=%.3f'% (plot_label, pvalue, np.sign(slope)*np.sqrt(R2))
    else:
        plot_label = '%s r=%.3f'% (plot_label, np.sign(slope)*np.sqrt(R2))
    ax.scatter(x, y, color=color, label=plot_label, s=markersize)
    ax.plot(temp, temppred, color=linecolor)
    return ax, slope, pvalue, R2


def make_silhouette_plot(X, cluster_labels):

    """
    
    
    """
    
    n_clusters = len(set(cluster_labels))
    
    fig, ax = plt.subplots(1, 1)
    fig.set_size_inches(4, 4)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax.set_xlim([-0.4, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax.set_ylim([0, len(X) + (n_clusters + 1) * 10])
    silhouette_avg = silhouette_score(X, cluster_labels, metric='cosine')

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels, metric='cosine')

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = colors_for_cluster[i]
        ax.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.9)

        # Label the silhouette plots with their cluster numbers at the middle
        ax.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i+1))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax.set_title("The silhouette plot for the various clusters.")
    ax.set_xlabel("The silhouette coefficient values")
    ax.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax.set_yticks([])  # Clear the yaxis labels / ticks
    ax.set_xticks([-0.4, -0.2, 0, 0.2, 0.4, 0.6, 0.8, 1])

In [ ]:
"""Ken Analysis"""

# Load the data
# basedir = r'\\172.25.144.34\LabCommon\Ken\data\2pImaging\result\result_pickle' # folder containing code and data
# populationdata = np.load(os.path.join(basedir, 'normal_cycle_female_23062020_140612_Alignedtotrial_meansignal_norm_sucrose_low_high estrogen.npy'),allow_pickle = True)
# print(str(populationdata.shape[0])+" cells were analyzed")


# dt_string = 'normal_cycle_female_23062020_140612_'
# normalization = 'mean'
# signal_type = "signal_norm" #'signal_norm'  
# analysis_condition = ''

In [ ]:
fname = '43384-1 400pwr 1104um 16x obj water-004'
fdir = r'D:\bruker_data\Sean\43384-1 400pwr 1104um 16x obj water-004'

save_dir = os.path.join(fdir, 'event_rel_analysis', 'clustering')

# import data and define trial types
data_dict_path = os.path.join(fdir, 'event_rel_analysis\event_data_dict.pkl')
with open(data_dict_path, 'rb') as pkl_handle:
    data_dict = pickle.load(pkl_handle)
trial_types =  data_dict.keys() #['low estrogen', 'high estrogen'] 

with open(os.path.join(fdir, 'event_analysis_fparam')) as json_file: 
    event_analysis_fparam = json.load(json_file) 
framerate = event_analysis_fparam['fs']
second_event_seconds = 1

# concatenates data across trials in the time axis; populationdata dimentionss are ROI by time (trials are appended)
populationdata = np.concatenate([data_dict[condition]['ztrial_avg_data'] for condition in trial_types], axis=1)
num_samples_trial = populationdata.shape[-1]/len(trial_types)

nan_rows = np.unique(np.where(np.isnan(populationdata))[0])
if nan_rows.size != 0:
    populationdata = np.delete(populationdata, obj=nan_rows, axis=0)
    print('Some ROIs contain nan in tseries!')
num_rois = populationdata.shape[0]


## Declaration of parameters

In [ ]:
# variables for plotting
pre_window_size = event_analysis_fparam['trial_start_end'][0]*framerate # n seconds multipled by framerate. Corresponds to baseline period prior to cues/event.
flag_plot_reward_line = False
frames_to_reward = second_event_seconds*framerate 

# define time vector for converting samples in a trial to seconds
tvec = np.linspace(event_analysis_fparam['trial_start_end'][0], event_analysis_fparam['trial_start_end'][1], num_samples_trial)

#variables for clustering
max_n_clusters = 10# Maximum number of clusters expected. I already ran this with up to 20 clusters and know
# that the optimal number is 9. So, I am leaving this at 11. In your data, might be worth increasing this, but
# it will take more time to run.

'''In spectral clustering: get n nearest neighbors for each data point and create connectivity graph (affinity matrix)'''
possible_n_nearest_neighbors = np.array([3,5,10]) # This should be selected for each dataset
# appropriately. When 4813 neurons are present, the above number of nearest neighbors ([30,40,30,50,60]) provides a good sweep of the
# parameter space. But it will need to be changed for other data.

# calculated variables
window_size = populationdata.shape[1]/2 # Total number of frames plotted around a cue
sortwindow_frames = [int(np.round(time*framerate)) for time in event_analysis_fparam['event_sort_win']] # Sort responses between first lick and 10 seconds.
sortresponse = np.argsort(np.mean(populationdata[:,sortwindow_frames[0]:sortwindow_frames[1]], axis=1))[::-1]
# sortresponse corresponds to an ordering of the neurons based on their average response in the sortwindow

cmax = np.nanmax(np.abs([np.nanmin(populationdata), np.nanmax(populationdata)])) # Maximum colormap value. 

In [ ]:
fig, axs = plt.subplots(2,2,figsize=(3*2,3*2), sharex='all', sharey='row')

# loop through conditions and plot heatmaps of trial-avged activity
for t in range(len(trial_types)):
    
    ax = axs[0,t]

    plot_extent = [tvec[0], tvec[-1], num_rois, 0 ]
    im = utils.subplot_heatmap(ax, ' ', populationdata[sortresponse, t*window_size: (t+1)*window_size], 
                               clims = [-cmax, cmax], extent_=plot_extent)
    axs[0,t].set_title(trial_types[t])
    
    ax.axvline(0, linestyle='--', color='k', linewidth=0.5)   
    if flag_plot_reward_line:
        ax.axvline(0, linestyle='--', color='k', linewidth=0.5)    
    
    ### roi-avg tseries 
    ax = axs[1,t]
    sns.tsplot(populationdata[sortresponse, t*window_size:(t+1)*window_size],
               ax=ax, time=tvec)
    ax.axvline(0, linestyle='--', color='k', linewidth=0.5)  
    if flag_plot_reward_line:
        ax.axvline(second_event_seconds, linestyle='--', color='k', linewidth=0.5)   
    ax.set_xlabel('Time from event (s)')   
 
axs[0,0].set_ylabel('Neurons')
axs[1,0].set_ylabel('Mean norm. fluor.')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
cbar = fig.colorbar(im, ax = axs, shrink = 0.7)
cbar.ax.set_ylabel('Heatmap Z-Score Activity', fontsize=13);

# fig.savefig(os.path.join(save_dir, 'results', tempstr+'.pdf'), format='pdf')
#fig.savefig(os.path.join(save_dir, dt_string+'_'+clusterkey + '+' + trial_types[0] + '_'+trial_types[1]+'.png'), format='png', dpi=300)

## Do PCA to reduce dimensionality in the time-domain

PCA: A linear algebra-based method to optimize how a set of variables can explain the variability of a dataset. Optimizing: meaning finding a new set of axes (ie. variables) that are linear combinations of the original axes where each new axis attempts to capture the most amount of variability in the data as possible while remaining linearly independent from the other new axes.

In this case, we are finding a new linearly independent parameter space that maximizes the explained variance into the top new axes

In [ ]:
def num_pc_explained_var(explained_var, explained_var_thresh=90):
    cum_sum = 0
    for idx, PC_var in enumerate(explained_var):
        cum_sum += PC_var
        if cum_sum > explained_var_thresh:
            return idx+1

In [ ]:
load_savedpca_or_dopca = 'dopca'
# Select 'dopca' for doing PCA on the data. Select 'savedpca' for loading my previous results

# perform PCA across time
if load_savedpca_or_dopca == 'dopca':
    pca = PCA(n_components=min(populationdata.shape[0],populationdata.shape[1]), whiten=True)
    pca.fit(populationdata) 
    with open(os.path.join(basedir, 'pcaresults.pickle'), 'wb') as f:
        pickle.dump(pca, f)
elif load_savedpca_or_dopca == 'savedpca':
    with open(os.path.join(basedir, 'OFCCaMKII_pcaresults.pickle'), 'rb') as f:
        pca = pickle.load(f)

# pca across time
transformed_data = pca.transform(populationdata)
# transformed data: each ROI is now a linear combination of the original time-serie
#np.save(os.path.join(save_dir, dt_string+'_'+clusterkey+'_' + "transformed_data.npy"),transformed_data)

# grab eigenvectors; linear combination of original axes
pca_vectors = pca.components_
print 'Number of PCs = %d'%(pca_vectors.shape[0])

# Number of PCs to be kept is defined as the number at which the 
# scree plot bends. This is done by simply bending the scree plot
# around the line joining (1, variance explained by first PC) and
# (num of PCs, variance explained by the last PC) and finding the 
# number of components just below the minimum of this rotated plot
x = 100*pca.explained_variance_ratio_ # eigenvalue ratios
xprime = x - (x[0] + (x[-1]-x[0])/(x.size-1)*np.arange(x.size))

# define number of PCs
num_retained_pcs = np.argmin(xprime) #num_pc_explained_var(x, 90) #
print 'Number of PCs to keep = %d'%(num_retained_pcs)

fig, ax = plt.subplots(figsize=(2,2))
ax.plot(np.arange(pca.explained_variance_ratio_.shape[0]).astype(int)+1, x, 'k')
ax.set_ylabel('Percentage of\nvariance explained')
ax.set_xlabel('PC number')
ax.axvline(num_retained_pcs, linestyle='--', color='k', linewidth=0.5)
ax.set_title('Scree plot')
# ax.set_xlim([0,50])
[i.set_linewidth(0.5) for i in ax.spines.itervalues()]
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

fig.subplots_adjust(left=0.3)
fig.subplots_adjust(right=0.98)
fig.subplots_adjust(bottom=0.25)
fig.subplots_adjust(top=0.9)
#fig.savefig(os.path.join(save_dir, dt_string+'_'+clusterkey+'_' + trial_types[0] + '_'+trial_types[1]+'_scree_plot.png'), format='png', dpi=300)


colors_for_key = {}
colors_for_key[trial_types[0]] = (0,0.5,1)
colors_for_key[trial_types[1]] = (1,0.5,0)

### plot retained principal components
numcols = 2.0
fig, axs = plt.subplots(int(np.ceil(num_retained_pcs/numcols)), int(numcols), sharey='all',
                        figsize=(2.2*numcols, 2.2*int(np.ceil(num_retained_pcs/numcols))))
for pc in range(num_retained_pcs):
    ax = axs.flat[pc]
    for k, tempkey in enumerate(trial_types):
        ax.plot(tvec, pca_vectors[pc, k*window_size:(k+1)*window_size], color=colors_for_key[tempkey],
                label='PC %d: %s'%(pc+1, tempkey))
        
    ax.axvline(0, linestyle='--', color='k', linewidth=1)
    ax.set_title('PC %d'%(pc+1))
    
    # labels
    if pc == 0:
        ax.set_xlabel('Time from cue (s)')
        ax.set_ylabel( 'PCA weights')


fig.tight_layout()
for ax in axs.flat[num_retained_pcs:]:
    ax.set_visible(False)

plt.tight_layout()
#fig.savefig(os.path.join(save_dir, dt_string+'_'+clusterkey+'_' +  trial_types[0] + '_'+trial_types[1]+'_PCA.png'), format='png', dpi=300)

## Spectral Clustering

In [ ]:
# calculate optimal number of clusters and nearest neighbors using silhouette scores
min_clusters = np.min([max_n_clusters+1, int(populationdata.shape[0])])
possible_n_clusters = np.arange(2, max_n_clusters+1) #This requires a minimum of 2 clusters.
# When the data contain no clusters at all, it will be quite visible when inspecting the two obtained clusters, 
# as the responses of the clusters will be quite similar. This will also be visible when plotting the data in
# the reduced dimensionality PC space (done below).

    
silhouette_scores = np.nan*np.ones((possible_n_clusters.size,
                                    possible_n_nearest_neighbors.size))

# loop through iterations of spectral clustering params
for n_clustersidx, n_clusters in enumerate(possible_n_clusters):
    for nnidx, nn in enumerate(possible_n_nearest_neighbors):
        model = SpectralClustering(n_clusters=n_clusters, affinity='nearest_neighbors', n_neighbors=nn)
        model.fit(transformed_data[:,:num_retained_pcs])
        silhouette_scores[n_clustersidx, nnidx] = silhouette_score(transformed_data[:,:num_retained_pcs],
                                                                   model.labels_,
                                                                   metric='cosine')
        print 'Done with numclusters = %d, num nearest neighbors = %d: score = %.3f'%(n_clusters,
                                                                                      nn,
                                                                                      silhouette_scores[n_clustersidx,                                                                           
                                                                                                        nnidx])

print 'Done with model fitting'

silhouette_dict = {}
silhouette_dict['possible_n_clusters'] = possible_n_clusters
silhouette_dict['possible_n_nearest_neighbors'] = possible_n_nearest_neighbors
silhouette_dict['silhouette_scores'] = silhouette_scores
silhouette_dict['shape'] = 'cluster_nn'
#with open(os.path.join(save_dir,dt_string+'_'+ clusterkey+'_' +  'silhouette_scores.pickle'), 'wb') as f:
#    pickle.dump(temp, f)

## Recluster with optimal params

In [ ]:
# Identify optimal parameters from the above parameter space
temp = np.where(silhouette_dict['silhouette_scores']==np.nanmax(silhouette_dict['silhouette_scores']))
n_clusters = silhouette_dict['possible_n_clusters'][temp[0][0]]
n_nearest_neighbors = silhouette_dict['possible_n_nearest_neighbors'][temp[1][0]]

print n_clusters, n_nearest_neighbors

# Redo clustering with these optimal parameters
model = SpectralClustering(n_clusters=n_clusters,
                           affinity='nearest_neighbors',
                           n_neighbors=n_nearest_neighbors)

# model = KMeans(n_clusters=n_clusters)

# model = AgglomerativeClustering(n_clusters=9,
#                                 affinity='l1',
#                                 linkage='average')

model.fit(transformed_data[:,:num_retained_pcs])

temp = silhouette_score(transformed_data[:,:num_retained_pcs], model.labels_, metric='cosine')

print 'Number of clusters = %d, average silhouette = %.3f'%(len(set(model.labels_)), temp)

# Save this optimal clustering model.
# with open(os.path.join(save_dir, 'clusteringmodel.pickle'), 'wb') as f:
#     pickle.dump(model, f)

          
# Since the clustering labels are arbitrary, I rename the clusters so that the first cluster will have the most
# positive response and the last cluster will have the most negative response.
def reorder_clusters(data, sort_win_frames, rawlabels):
    uniquelabels = list(set(rawlabels))
    responses = np.nan*np.ones((len(uniquelabels),))
    for l, label in enumerate(uniquelabels):
        responses[l] = np.mean(data[rawlabels==label, sort_win_frames[0]:sort_win_frames[1]])
    temp = np.argsort(responses).astype(int)[::-1]
    temp = np.array([np.where(temp==a)[0][0] for a in uniquelabels])
    outputlabels = np.array([temp[a] for a in list(np.digitize(rawlabels, uniquelabels)-1)])
    return outputlabels
newlabels = reorder_clusters(populationdata, sortwindow_frames, model.labels_)

# Create a new variable containing all unique cluster labels
uniquelabels = list(set(newlabels))

# np.save(os.path.join(summarydictdir, dt_string+'_'+ clusterkey+'_' + 'spectral_clusterlabels.npy'), newlabels)

colors_for_cluster = [[0.933, 0.250, 0.211],
                      [0.941, 0.352, 0.156],
                      [0.964, 0.572, 0.117],
                      [0.980, 0.686, 0.250],
                      [0.545, 0.772, 0.247],
                      [0.215, 0.701, 0.290],
                      [0, 0.576, 0.270],
                      [0, 0.650, 0.611],
                      [0.145, 0.662, 0.878]]

In [ ]:

sortwindow = [15, 100]

fig, axs = plt.subplots(len(trial_types),len(uniquelabels),
                        figsize=(2*len(uniquelabels),2*len(trial_types)))
if len(axs.shape) == 1:
    axs = np.expand_dims(axs, axis=0)

numroisincluster = np.nan*np.ones((len(uniquelabels),))

for c, cluster in enumerate(uniquelabels):
    for k, tempkey in enumerate(trial_types):
        temp = populationdata[np.where(newlabels==cluster)[0], k*window_size:(k+1)*window_size]
        numroisincluster[c] = temp.shape[0]
        ax=axs[k, cluster]
        sortresponse = np.argsort(np.mean(temp[:,sortwindow[0]:sortwindow[1]], axis=1))[::-1]
        
        plot_extent = [tvec[0], tvec[-1], len(sortresponse), 0 ]
        im = utils.subplot_heatmap(ax, ' ', temp[sortresponse], 
                                   clims = [-cmax, cmax], extent_=plot_extent)

        axs[k, cluster].grid(False) 
        if k!=len(trial_types)-1:

            axs[k, cluster].set_xticks([])

        axs[k, cluster].set_yticks([])
        axs[k, cluster].axvline(0, linestyle='--', color='k', linewidth=0.5)
        if flag_plot_reward_line:
            axs[k, cluster].axvline(second_event_seconds, linestyle='--', color='k', linewidth=0.5)
        if cluster==0:
            axs[k, 0].set_ylabel('%s'%(tempkey))
    axs[0, cluster].set_title('Cluster %d\n(n=%d)'%(cluster+1, numroisincluster[c]))
    
fig.text(0.5, 0.05, 'Time from cue (s)', fontsize=12,
         horizontalalignment='center', verticalalignment='center', rotation='horizontal')
fig.tight_layout()

fig.subplots_adjust(wspace=0.1, hspace=0.1)
fig.subplots_adjust(left=0.03)
fig.subplots_adjust(right=0.93)
fig.subplots_adjust(bottom=0.2)
fig.subplots_adjust(top=0.83)                                    

cbar = fig.colorbar(im, ax = axs, shrink = 0.7)
cbar.ax.set_ylabel('Z-Score Activity', fontsize=13);

#fig.savefig(os.path.join(save_dir, dt_string+'_'+clusterkey+'_' +  trial_types[0] + '_'+trial_types[1]+'_clusters_spectral_clustering.png'),format='png', dpi=300,bbox_inches='tight')


In [ ]:
tvec_convert_dict = {}
for i in range(len(tvec)):
    tvec_convert_dict[i] = tvec[i] 

In [ ]:

fig, axs = plt.subplots(1,len(uniquelabels),
                        figsize=(3*len(uniquelabels),1.3*len(trial_types)))

for c, cluster in enumerate(uniquelabels):

    for k, tempkey in enumerate(trial_types):
        temp = populationdata[np.where(newlabels==cluster)[0], k*window_size:(k+1)*window_size]
        numroisincluster[c] = temp.shape[0]
        sortresponse = np.argsort(np.mean(temp[:,sortwindow[0]:sortwindow[1]], axis=1))[::-1]
        sns.lineplot(x="variable", y="value",data = pd.DataFrame(temp[sortresponse]).rename(columns=tvec_convert_dict).melt(),
                    ax = axs[cluster],
                    palette=plt.get_cmap('coolwarm'),label = tempkey,legend = False)
        axs[cluster].grid(False)  
        axs[cluster].axvline(0, linestyle='--', color='k', linewidth=0.5)
#         axs[cluster].axvline(pre_window_size + frames_to_reward, linestyle='--', color='k', linewidth=0.5)
        axs[cluster].spines['right'].set_visible(False)
        axs[cluster].spines['top'].set_visible(False)
        if cluster==0:
            axs[cluster].set_ylabel('Normalized fluorescence')
        else:
            axs[cluster].set_ylabel('')
        axs[cluster].set_xlabel('')
    axs[cluster].set_title('Cluster %d\n(n=%d)'%(cluster+1, numroisincluster[c]))
    axs[0].legend()
fig.text(0.5, 0.05, 'Time from cue (s)', fontsize=12,
         horizontalalignment='center', verticalalignment='center', rotation='horizontal')
fig.tight_layout()

fig.subplots_adjust(wspace=0.1, hspace=0.1)
fig.subplots_adjust(left=0.03)
fig.subplots_adjust(right=0.93)
fig.subplots_adjust(bottom=0.2)
fig.subplots_adjust(top=0.83)

#fig.savefig(os.path.join(save_dir, dt_string+'_'+ clusterkey+'_' + trial_types[0] + '_'+trial_types[1]+'_clusters_average_spectral_clustering.png'), format='png', dpi=300,bbox_inches='tight')

In [ ]:
num_clusterpairs = len(uniquelabels)*(len(uniquelabels)-1)/2

numrows = int(np.ceil(num_clusterpairs**0.5))
numcols = int(np.ceil(num_clusterpairs/np.ceil(num_clusterpairs**0.5)))
fig, axs = plt.subplots(numrows, numcols, figsize=(3*numrows, 3*numcols))

tempsum = 0
for c1, cluster1 in enumerate(uniquelabels):
    for c2, cluster2 in enumerate(uniquelabels):
        if cluster1>=cluster2:
            continue
        temp1 = transformed_data[np.where(newlabels==cluster1)[0], :num_retained_pcs]
        temp2 = transformed_data[np.where(newlabels==cluster2)[0], :num_retained_pcs]
        X = np.concatenate((temp1, temp2), axis=0)
        tsne = TSNE(n_components=2, init='random',
                    random_state=0, perplexity=100)
        Y = tsne.fit_transform(X)
        if numrows*numcols==1:
            ax = axs
        else:
            ax = axs[tempsum/numcols,
                     tempsum - tempsum/numcols*numcols]   
        ax.scatter(Y[:np.sum(newlabels==cluster1),0],
                   Y[:np.sum(newlabels==cluster1),1],
                   color=colors_for_cluster[cluster1], label='Cluster %d'%(cluster1+1), alpha=1)
        ax.scatter(Y[np.sum(newlabels==cluster1):,0],
                   Y[np.sum(newlabels==cluster1):,1],
                   color=colors_for_cluster[cluster2+3], label='Cluster %d'%(cluster2+1), alpha=1)

        ax.set_xlabel('tsne dimension 1')
        ax.set_ylabel('tsne dimension 2')
        ax.legend()
        tempsum += 1

        fig.tight_layout()